In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
import numpy as np
import random
import argparse
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, random_split
import utils
import torch.optim.lr_scheduler as lr_scheduler


import models
import training

NUM_CLASSES = 10


### Self-Supervised MNIST

In [7]:
# MNIST TRAINING

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root="./data", train=True, download=False, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=False, transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
dl_train = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
dl_val = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)
dl_test = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model = models.SelfSupervised_MNIST(latent_dim=128)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

checkpoints_path = "encoder_mnist.pth"
trainer = training.SelfSupervisedTrainer(model, dl_train, dl_val, loss_fn, optimizer, device)
trainer.train(num_epochs=10, checkpoints=checkpoints_path)

KeyboardInterrupt: 

### Self-Supervised MNIST: Reconstruction Results

In [ ]:
indices = np.random.choice(len(test_dataset), 5, replace=False)

utils.showReconstructions(model, test_dataset, device, indices)


### Self-Supervised MNIST: Linear Interpolation


In [ ]:
utils.displayInterpolation(model, test_dataset, device, indices[0], indices[4])

### Self-Supervised MNIST: t-SNE 

In [ ]:

utils.plot_tsne(model.encoder, dl_test, device)

In [ ]:
utils.display_tsne()


In [ ]:
encoder = models.SelfSupervised_MNIST(latent_dim=128).encoder
encoder.load_state_dict(torch.load("encoder_mnist.pth"))
encoder.to(device)
encoder.eval()  

classifier = models.Classifier(latent_dim=128, num_classes=10).to(device)
fn_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

checkpoints_path = "classifier_mnist.pth"
trainer = training.ClassifierTrainer(classifier, encoder, dl_train, dl_val, fn_loss, optimizer, device)
trainer.train(num_epochs=10, checkpoints=checkpoints_path)

### Classification-Guided MNIST

In [ ]:
model = models.ClassificationGuided_MNIST().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()
trainer = training.ClassificationGuidedTrainer(model,  dl_train, dl_val, dl_test, loss_fn, optimizer, device)
checkpoints_path = "guided_mnist.pth"

trainer.train(num_epochs=10, checkpoints=checkpoints_path)

### Classification-Guided MNIST: t-SNE

In [ ]:
utils.plot_tsne(model.encoder, dl_test, device)

In [ ]:
utils.display_tsne()


### Self-Supervised CIFAR10

In [ ]:
# CIFAR10 TRAINING
'''
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
'''
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])])
train_dataset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=False)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=False)

train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
dl_train = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
dl_val = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True)
dl_test = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.SelfSupervised_CIFAR10(latent_dim=128)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

checkpoints_path = "encoder_cifar10.pth"
trainer = training.SelfSupervisedTrainer(model, dl_train, dl_val, loss_fn, optimizer, device, scheduler)
trainer.train(num_epochs=20, checkpoints=checkpoints_path)

### Self-Supervised CIFAR10: Reconstruction Results

In [ ]:

indices = np.random.choice(len(test_dataset), 5, replace=False)

utils.showReconstructions(model, test_dataset, device, indices)

In [ ]:
encoder = models.SelfSupervised_CIFAR10(latent_dim=128).encoder
encoder.load_state_dict(torch.load("encoder_cifar10.pth"))
encoder.to(device)
encoder.eval()  

classifier = models.Classifier(latent_dim=128, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


transform = transforms.Compose([transforms.ToTensor()])

checkpoints_path = "classifier_cifar10.pth"
trainer = training.ClassifierTrainer(classifier, encoder, dl_train, dl_test, criterion, optimizer, device, scheduler)
trainer.train(num_epochs=30, checkpoints=checkpoints_path)

### Self-Supervised CIFAR10: t-SNE 

In [ ]:
utils.plot_tsne(model.encoder, dl_test, device)

In [ ]:
utils.display_tsne()

### Classification-Guided CIFAR10

In [ ]:

model = models.ClassificationGuided_CIFAR10().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

loss_fn = nn.CrossEntropyLoss()
trainer = training.ClassificationGuidedTrainer(model,  dl_train, dl_val, dl_test, loss_fn, optimizer, device)
checkpoints_path = "guided_cifar10.pth"

trainer.train(num_epochs=50, checkpoints=checkpoints_path)

### Classification-Guided CIFAR10: t-SNE

In [ ]:
utils.plot_tsne(model.encoder, dl_test, device)

In [ ]:
utils.display_tsne()

In [6]:
train_dataset = datasets.CIFAR10(root="/datasets/cv_datasets/data", train=True, download=True)
test_dataset = datasets.CIFAR10(root="/datasets/cv_datasets/data", train=False, download=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
epochs = 100

clr_lr = 0.05
class_lr = 0.001

T = 0.1
batch_size = 1024

In [7]:
from ConstructiveNet import *
from models import simCLR_CIFAR10
from training import simCLRTrainer
from utils import create_data_sets

clr_encoder = Encoder() 
clr_projection_head = ProjectionHead()
clr_classifier = LinearClassifier()
model = simCLR_CIFAR10(clr_encoder, clr_projection_head, clr_classifier).to(device)
loss_fn = NTXentLoss
class_loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=clr_lr)
class_optimizer = optim.Adam(model.classifier_head.parameters(), lr=class_lr)

trainer = simCLRTrainer(epochs=epochs, model=model, optimizer=optimizer, loss_fn=loss_fn, temperature=T, device=device)

In [9]:
simclr_transform = get_simclr_augmentation(input_size=32)

train_ds = SimCLRDataset(train_dataset, transform=simclr_transform)
test_ds = TestDataset(test_dataset, transform=simclr_transform)

dl_train, dl_valid, dl_test = create_data_sets(train_ds, test_ds, 0.1, batch_size, simclr_transform)

trainer.fit(dl_train, dl_valid)

Training set size: 45000
Validation set size: 5000
Test set size: 10000
------Epoch [1/20]:--------


Training Loss: 6.0702  |  Validation Loss: 0.6773
------Epoch [2/20]:--------


Training Loss: 6.0424  |  Validation Loss: 0.6723
------Epoch [3/20]:--------


KeyboardInterrupt: 

In [5]:
for param in trainer.model.encoder.parameters():
    param.requires_grad = False

classifier_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_ds_classifier = TestDataset(train_dataset, classifier_transform)
train_loader_classifier = torch.utils.data.DataLoader(train_ds_classifier, batch_size=256)

trainer.train_classifier(train_loader_classifier, class_optimizer, class_loss_fn)

trainer.test(dl_test)

Epoch 1, Loss: 0.0066


Epoch 2, Loss: 0.0056


Epoch 3, Loss: 0.0054


Epoch 4, Loss: 0.0053


Epoch 5, Loss: 0.0052


...

Epoch 17, Loss: 0.0050


Epoch 18, Loss: 0.0050


Epoch 19, Loss: 0.0050


Epoch 20, Loss: 0.0050


Test Accuracy =  30.23%
